In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import networkx as nx


%matplotlib inline

Modelo SEAIR
============

No modelo **SEAIR** temos que:

1. S = número de sucetíveis 
2. E = número de expostos
3. A = número de assintomáticos
4. I = número de infectados
5. R = número de recuperados

*P é População da cidade*

---

Um sucetível passa para exposto quando em contato com um assintomático (probabilidade $\beta_A$) ou em contato com um infectado (probabilidade $\beta_I$). 

Supondo que um agente entre em **contato em média com $k$** pessoas por dia, então 

1. $S(t+1)=S(t)-\Pi S(t)$
2. $E(t+1)=E(t)+\Pi S(t)-\eta E(t)$
3. $A(t+1)=A(t)+\eta E(t)-\alpha A(t)$
4. $I(t+1)=I(t)+\alpha A(t)-\mu I(t)$
5. $R(t+1)=R(t)+\mu I(t)$

Onde $\Pi$ é a probabilidade de um agente ser infectado:

$$\Pi=1-(1-\beta_I)^{k I(t)/P}(1-\beta_A)^{k A(t)/P}$$
---
---

1) Implementar uma função chamada **Epidemic_OneCity(N,k,\beta_I,\beta_A,\eta,\alpha,\mu)**, com a condição inicial: 
1. $S(0)=1.0-0.01$
2. $E(0)=0.0$
3. $A(0)=0.01$
4. $I(0)=0.0$
5. $R(0)=0.0$

Retornar a evolução temporal de $S(t)$, $E(t)+A(t)+I(t)$, e $R(t)$ e fazer uma gráfico para elas.

Como muda $E(t)+A(t)+I(t)$ para diferentes $k$?

Usar:
1. $\beta_I=0.06$
2. $\beta_A=0.06$
3. $\eta=1/2.32$
4. $\alpha=1/2.86$
5. $\mu=1/3.2$



In [2]:
def Epidemic_OneCity(N,k,beta_I,beta_A,eta,alpha,mu):
    return

Modelo SEAIR com duas cidades
============

Expandir o modelo **SEAIR** para duas cidades é necessário ter em conta a mobilidade entre elas, dado pela matrix OD:

$$ R= \begin{pmatrix} R_{00} & R_{01} \\ R_{10} & R_{11} \end{pmatrix}$$


$R_{ij}$ mede a fração da população que vai da cidade $i$ até a cidade $j$ todos os dias.

* A mobilidade promove uma **MISTURA** entre os habitantes das duas cidades.

* Todos os compartimentos e a população agora são vetores: S[0,1], E[0,1] .... N[0,1]

Cada cidade agora tem uma população efetiva:
$$P_{eff}[0]=R_{00}*P[0] + R_{10}*P[1]$$    
$$P_{eff}[1]=R_{10}*P[0] + R_{11}*P[1]$$

E o número de infectados e assintomáticos que em média uma pessoa da cidade $0$ pode encontrar é 
$$I_{eff}[0]=I[0]*R_{00}+I[1]*R_{10}$$

Logo a probabilidade de infecção é:

$$p[0](t)=1-(1-\beta_I)^{k I_{eff}[0](t)/P_{eff}}(1-\beta_A)^{k A_{eff}[0](t)/P_{eff}}$$
---
e finalmente:

$$\Pi[i]=\sum_j R_{ij}p_{j}$$
---

Através desse novo $\Pi_i$ podemos integrar todas as equações:

1. $S_i(t+1)=S_i(t)-\Pi_i S_i(t)$
2. $E_i(t+1)=E_i(t)+\Pi_i S_i(t)-\eta E_i(t)$
3. $A_i(t+1)=A_i(t)+\eta E_i(t)-\alpha A_i(t)$
4. $I_i(t+1)=I_i(t)+\alpha A_i(t)-\mu I_i(t)$
5. $R_i(t+1)=R_i(t)+\mu I_i(t)$

1) Usando a matrix OD

$$ R= \begin{pmatrix} 0.9 & 0.1 \\ 0.0 & 1.0 \end{pmatrix},$$

população $P=[500000,50000]$, e $100$ **Assintomáticos** no tempo zero na primeira cidade. 

### Qual é a evolução da epidemia para as duas cidades?


# Estender o código para N cidades

* **Usar a notação de matrizes do numpy o máximo possível**
---

* Uma vez estendido para N cidade, podemos simular para os 308 concelhos de Portugal;

* O dados demográficos estão no arquivo concelho_info.csv:


1. ID = Código de identificação do concelho;
2. Nome = Nome do concelho;
3. PopTot = População do concelho;
4. Superfície = Área do concelho;
5. NUT = Região do concelho;

*Só iremos aqui usar as colunas ID, Nome, PopTot*

* Leia o concelho_info.csv usando
```Python
df_concelhos = pd.read_csv("concelho_info.csv",sep=",")```

* Guarde o vetor de IDs e Populações
```Python
Pop=np.array(df_concelhos.PopTot)
ID=np.array(df_concelhos.ID)```

* Ler arquivo matriz_OD.dat com a matriz OD de Portugal
```Python
def read_matrixOD(N):
    file=open("Matriz_OD.txt","r")
    
    R=np.zeros((N,N))
    for line in file:
        data=line.split(" ")
        i=int(data[0])
        j=int(data[1])
        R[i,j]=float(data[-1])
    return R```
    
* Colocar 10 casos **Assintomáticos** em Lisboa como condição inicial

### Qual a evolução no número de casos em Portugal? Em Lisboa? No Porto?


# Visualização da evolução do número de caso em um mapa

Primeiro é necessário carregar o shapefile do mapa


```Python
fp = "concelhos-shapefile/concelhos.shp"
map_df = gpd.read_file(fp)
map_df.CCA_2=map_df.CCA_2.astype(int)```

Para se fazer um **mapa coroplético** é necessário primeiro definir a variável a ser transformada em uma cor. Vamos usar risco $= A_i(t)+E_i(t)+I_i(t)$. 

* Construa um dicionário risco[i], onde i é o ID do concelho.
* Junte esse dicionário ao map_df usando o merge:
```Python
merged = map_df.merge(risco, how='left', left_on="CCA_2", right_on="ID")```

* Para plotar o mapa é necessário primeiro definir uma escala de cores e uma normalização
```Python
vmin, vmax = min(populacao.Populacao), max(populacao.Populacao)
fig, ax = plt.subplots(1, figsize=(30, 10))
ax.axis('off')
"""Onde está Portugal Continental? e as ilhas?"""
plt.xlim(?,?)
plt.ylim(?,?)
cmap = plt.cm.get_cmap("inferno_r")
norm = mpl.colors.SymLogNorm(0.0001, vmin=vmin, vmax=vmax)
sm = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
fig.colorbar(sm)```

* Por fim 
```Python
merged.plot(column="Populacao", cmap=cmap, vmin=vmin,vmax=vmax, linewidth=0.8, ax=ax, edgecolor='0.1')
plt.savefig("mapa_dia%s.png"%dia,bbox_inches='tight',dpi=100)```

---

## Como evolui no tempo? Qual a diferença de começar em Lisboa ou Campo Maior?

